In [1]:
import os
import pandas as pd
import numpy as np
import neurokit2 as nk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from tqdm import tqdm

# Constants
ecg_folder = "../../../../Datasets/12-lead electrocardiogram database/ECGDataDenoised"
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

# Load diagnostics data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)


# Helper function to preprocess and extract features using NeuroKit2
def preprocess_ecg(ecg_signal, sampling_rate=500):
    try:
        # Process the ECG signal
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=sampling_rate)

        # Extract NeuroKit2-calculated features
        features = signals.mean().to_dict()
        return features
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None


# Process files and extract features
data = []
labels = []
errors = []

for idx, row in tqdm(diagnostics_df.iterrows(), total=len(diagnostics_df)):
    ecg_file = os.path.join(ecg_folder, row["FileName"] + ".csv")
    if os.path.exists(ecg_file):
        try:
            lead_ii = pd.read_csv(ecg_file, header=0).iloc[:, 1].values  # Extract Lead II data
            features = preprocess_ecg(lead_ii)
            if features is not None:
                data.append(features)
                labels.append(row["Rhythm"])
            else:
                errors.append((row["FileName"], "Feature extraction failed"))
        except Exception as e:
            errors.append((row["FileName"], str(e)))
    else:
        errors.append((row["FileName"], "File not found"))

# Log errors
if errors:
    print("Files with errors:")
    for file, error in errors:
        print(f"{file}: {error}")

# Convert to DataFrame
data = pd.DataFrame(data)
labels = pd.Series(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

2024-12-03 17:18:49.597689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 17:18:49.608356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 17:18:49.611599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 17:18:49.620567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 17:18:50.167107: W tensorflow/compiler/tf2

Error during preprocessing: cannot convert float NaN to integer


 10%|▉         | 1063/10646 [01:11<11:03, 14.43it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
 10%|█         | 1066/10646 [01:12<10:00, 15.94it/s]

Error during preprocessing: cannot convert float NaN to integer


 12%|█▏        | 1308/10646 [01:28<10:13, 15.22it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
 14%|█▍        | 1474/10646 [01:39<10:11, 14.99it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neuroki

...

190/265 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.5392 - loss: 1.1247

I0000 00:00:1733225552.590316  366949 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


265/265 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5769 - loss: 1.0519
Epoch 2/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7984 - loss: 0.5630
Epoch 3/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.8249 - loss: 0.4869
Epoch 4/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.8466 - loss: 0.4474
Epoch 5/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - accuracy: 0.8440 - loss: 0.4281
Epoch 6/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 0.8571 - loss: 0.4080
Epoch 7/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.8671 - loss: 0.3882
Epoch 8/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.8657 - loss: 0.3925
Epoch 9/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - accuracy: 0.8648 - loss: 0.3871
Epoch 10/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.8749 - loss: 0.3677
Epoch 11/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.8830 - loss: 0.3409
Epoch 12/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


265/265 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6514 - loss: 0.8785
Epoch 2/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8320 - loss: 0.5037
Epoch 3/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8540 - loss: 0.4246
Epoch 4/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.8663 - loss: 0.3956
Epoch 5/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.8705 - loss: 0.3966
Epoch 6/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - accuracy: 0.8660 - loss: 0.3902
Epoch 7/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.8755 - loss: 0.3726
Epoch 8/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.8729 - loss: 0.3704
Epoch 9/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.8751 - loss: 0.3608
Epoch 10/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.8819 - loss: 0.3368
Epoch 11/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.8805 - loss: 0.3438
Epoch 12/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 663

In [ ]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=label_encoder.classes_))

# MLP Model
mlp_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
y_pred_mlp = np.argmax(mlp_model.predict(X_test), axis=1)
print("MLP Classification Report:")
print(classification_report(y_test, y_pred_mlp, target_names=label_encoder.classes_))

# CNN Model
cnn_model = Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
y_pred_cnn = np.argmax(cnn_model.predict(X_test), axis=1)
print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn, target_names=label_encoder.classes_))
